$\mathbf{T}=\left[\begin{array}{cccc}T_{11} & T_{12} & \ldots & T_{1 M} \\ T_{21} & T_{22} & \ldots & T_{2 M} \\ \vdots & \vdots & \ddots & \vdots \\ T_{M 1} & T_{M 2} & \ldots & T_{M M}\end{array}\right]$

In [3]:
from __future__ import division
import math
import sys, site, os
import numpy as np
from scipy import optimize 
from scipy.special import comb
from scipy.stats import poisson
import scipy.optimize
import scipy.misc
from datetime import datetime
sys.path.append(os.path.abspath("../auxiliary_scripts/"))
from main_aux import *


########### Mask Model ES Analysis -- Parellel ########### 
def P_A_given_R(i, T_list, k0, k1):
    one_minus_T1 = 1 - T_list[0][1]
    one_minus_T2 = 1 - T_list[0][0]
    one_minus_T3 = 1 - T_list[1][1]
    one_minus_T4 = 1 - T_list[1][0]
    if i == 0:
        res = 1 - (one_minus_T2 ** k0) * (one_minus_T4 ** k1)
    else:
        res = 1 - (one_minus_T1 ** k0) * (one_minus_T3 ** k1)
    assert res >= 0, "P_A_given_R should be greater than 0"
    assert res <= 1, "P_A_given_R should be less than 1"
    return res

def P_A_given_B_N(i, is_intermediate, k, n, T_list, A_0, A_1):
    one_minus_A0 = 1 - A_0
    one_minus_A1 = 1 - A_1
    
    p_abn = 0
    
    if is_intermediate:
        k1_range = k - n
    else:
        k1_range = k + 1 - n
    
    for k0 in range(n + 1):
        for k1 in range(k1_range):
            p_a_given_r = P_A_given_R(i, T_list, k0, k1)
            p_abn += p_a_given_r * \
                     comb(n, k0) * \
                     comb(k1_range - 1, k1) * \
                     (A_0 ** k0) * \
                     (A_1 ** k1) * \
                     (one_minus_A0 ** (n - k0)) * \
                     (one_minus_A1 ** (k1_range - 1 - k1))
    return p_abn

def P_A_given_B(i, is_intermediate, k, T_list, A_0, A_1, m):
    p_ab = 0
    if is_intermediate:
        n_range = k
    else:
        n_range = k + 1
        
    for n in range(n_range):
        p_abn = P_A_given_B_N(i, is_intermediate, k, n, T_list, A_0, A_1)
        p_ab += p_abn * comb(n_range - 1, n) * \
                (m ** n) * \
                ((1 - m) ** (n_range - 1 - n))
    return p_ab

def P_A(i, is_intermediate, mean_degree, T_list, m, A, k_max):    
    pa_L = 0
    for k in range(1, k_max):
        prob_r = poisson.pmf(k, mean_degree)
        if is_intermediate: # intermediate q using excess degree distribution
            pb = prob_r * k * 1.0 / mean_degree 
        else:
            pb = prob_r
#         p_b = k * p_k / mean_degree
        p_ab = P_A_given_B(i, is_intermediate, k, T_list, A[0], A[1], m)
        pa_L += p_ab * pb
    return pa_L

def pA_vec(mean_degree, is_intermediate, T_list, m, A, k_max):
    A0 = P_A(0, is_intermediate, mean_degree, T_list, m, A, k_max)
    A1 = P_A(1, is_intermediate, mean_degree, T_list, m, A, k_max)
    return [A0, A1]


def func_root(A, mean_degree, T_list, m, k_max):
    return np.array(pA_vec(mean_degree, True, T_list, m, A, k_max)) - np.array(A)

def get_EpidemicSize(mean_degree, paras, infection_size0, infection_size1, infection_size):
    '''
    S
    '''    
    k_max, T_list = resolve_paras(paras)
    
    init_A = (0.9, 0.9)
    m = paras.m

    A_0_1_root = optimize.fsolve(func_root, init_A, args=(mean_degree, T_list, m, k_max))
    A0, A1 = pA_vec(mean_degree, False,  T_list, paras.m, A_0_1_root, k_max)
    A = A0 * paras.m + A1 * (1 - paras.m)
    print(mean_degree, A, A0, A1)
    infection_size0[mean_degree] = A0
    infection_size1[mean_degree] = A1
    infection_size[mean_degree]  = A
    return A0, A1, A